# 初始化数据和方法

In [7]:
import openai
import os

openai.api_key = os.environ.get('OPENAI_API_KEY')

def sent_gpt_request(text, retries=3):
    print("正在请求...")

    retries_total = 0
    while retries_total < retries:
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k-0613",
                messages=[
                    {"role": "user", "content": """你的任务是从一篇试卷中提取选择题的题目、选项、答案和解析，并按照特定的JSON格式返回。如果试卷没有答案、选项或解析，那么对应的字段应为空。如果试卷中没有选择题，应返回空列表。"""},
                    {"role": "assistant", "content": "我理解你的需求。你需要我从给定的试卷中提取选择题的题目、选项、答案和解析，并按照特定的JSON格式返回。如果试卷没有答案、选项或解析，那么对应的字段应为空。如果试卷中没有选择题，我应返回空列表。现在，请提供你需要的JSON格式。"},
                    {"role": "user", "content": """我需要的格式如下：\n [{"answer": "A" OR "", "option": {"A":"2倍","B":"4倍","C":"8倍"} OR "", "question": "15．圆柱的高扩大2倍，底面半径也扩大2倍，圆柱的体积就扩大（　　）", "explanation": "解析内容" OR ""}, ...]。请注意，选项字段中的'A'.'B'.'C'.'D'是作为键存在的，而不是必须填入的内容。"""},
                    {"role": "assistant", "content": "我明白了你的需求。现在，请提供你的试卷。"},
                    {"role": "user", "content":text},
                    {"role": "assistant", "content": '我已经从试卷中提取出选择题，并按照你指定的格式返回了结果。如果试卷中没有选择题，我返回了一个空列表。这是我找到的选择题的JSON数据:\n'}
                ],
                temperature=0,
            )
            return completion.choices[0]["message"]["content"]
        except Exception as e:
            retries_total += 1
            print(f"请求失败重新请求中... {e}")

    return None

In [23]:
[
    {"role": "user", "content": """你的任务是从一篇试卷中提取选择题的题目、选项、答案和解析，并按照特定的JSON格式返回。如果试卷没有答案、选项或解析，那么对应的字段应为空。如果试卷中没有选择题，应返回空列表。"""},
    {"role": "assistant", "content": "我理解你的需求。你需要我从给定的试卷中提取选择题的题目、选项、答案和解析，并按照特定的JSON格式返回。如果试卷没有答案、选项或解析，那么对应的字段应为空。如果试卷中没有选择题，我应返回空列表。现在，请提供你需要的JSON格式。"},
    {"role": "user", "content": """我需要的格式如下：\n [{"answer": "A" OR "", "option": {"A":"2倍","B":"4倍","C":"8倍"} OR "", "question": "15．圆柱的高扩大2倍，底面半径也扩大2倍，圆柱的体积就扩大（　　）", "explanation": "解析内容" OR ""}, ...]。请注意，选项字段中的'A'.'B'.'C'.'D'是作为键存在的，而不是必须填入的内容。"""},
    {"role": "assistant", "content": "我明白了你的需求。现在，请提供你的试卷。"},
    {"role": "user", "content":""},
    {"role": "assistant", "content": '我已经从试卷中提取出选择题，并按照你指定的格式返回了结果。如果试卷中没有选择题，我返回了一个空列表。这是我找到的选择题的JSON数据:\n'}
],

([{'role': 'user',
   'content': '你的任务是从一篇试卷中提取选择题的题目、选项、答案和解析，并按照特定的JSON格式返回。如果试卷没有答案、选项或解析，那么对应的字段应为空。如果试卷中没有选择题，应返回空列表。'},
  {'role': 'assistant',
   'content': '我理解你的需求。你需要我从给定的试卷中提取选择题的题目、选项、答案和解析，并按照特定的JSON格式返回。如果试卷没有答案、选项或解析，那么对应的字段应为空。如果试卷中没有选择题，我应返回空列表。现在，请提供你需要的JSON格式。'},
  {'role': 'user',
   'content': '我需要的格式如下：\n [{"answer": "A" OR "", "option": {"A":"2倍","B":"4倍","C":"8倍"} OR "", "question": "15．圆柱的高扩大2倍，底面半径也扩大2倍，圆柱的体积就扩大（\u3000\u3000）", "explanation": "解析内容" OR ""}, ...]。请注意，选项字段中的\'A\'.\'B\'.\'C\'.\'D\'是作为键存在的，而不是必须填入的内容。'},
  {'role': 'assistant', 'content': '我明白了你的需求。现在，请提供你的试卷。'},
  {'role': 'user', 'content': ''},
  {'role': 'assistant',
   'content': '我已经从试卷中提取出选择题，并按照你指定的格式返回了结果。如果试卷中没有选择题，我返回了一个空列表。这是我找到的选择题的JSON数据:\n'}],)

In [17]:
def per_process(text):
    liens = text.splitlines()
    liens = map(lambda x: x.replace("> ","").replace(">","").replace("*", ""), liens)
    liens = filter(lambda x: x.strip() != "", liens)
   
    return "\n".join(liens)

In [3]:
import os
import glob
from pathlib import Path

examination_paper_list= []

path = Path("./answer_markdown")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": file,
            "text": f.read()
        })

In [4]:
import random
import tiktoken

enc = tiktoken.encoding_for_model('gpt-3.5-turbo')


def generate_batch(text, max_token=10000):
    batch = []
    tmp = ""
    
    for line in text.splitlines():
        if len(enc.encode(tmp+line)) > max_token:
            batch.append(tmp)
            tmp = line
            continue
            
        tmp += "\n" + line
      
    batch.append(tmp)
    
    return batch

def random_generat_prompt(text_list):
    random_number = random.randint(0, len(text_list))
    
    return generate_batch(text_list[random_number])

# 获取数据

In [31]:
batch = random_generat_prompt([examination_paper["text"] for examination_paper in examination_paper_list])
prompt = batch[0]
print(len(batch))

1


In [32]:
prompt

'\n**2019-2020学年陕西省西安市陕师大附小四年级（上）期末数学试卷**\n\n**一、填空题（共22分）**\n\n1．（2分）2017年国庆黄金周，西安共接待游客11650300人次，横线上的数读作[\u3000 \u3000]{.underline}；旅游收入5795000000元人民币，四舍五入到亿位大约是[\u3000 \u3000]{.underline}亿元．\n\n2．（2分）黑板的上下两条边互相[\u3000 \u3000]{.underline}；相邻的两条边互相[\u3000 \u3000]{.underline}．\n\n3．（1分）钟面上[\u3000 \u3000]{.underline}时整的时候，时针和分针成平角．\n\n4．（4分）（*a*+*b*）+*c*＝[\u3000 \u3000]{.underline}+（[\u3000 \u3000]{.underline}+[\u3000 \u3000]{.underline}）；（*a*+*b*）×*c*＝[\u3000 \u3000]{.underline}×[\u3000 \u3000]{.underline}+[\u3000 \u3000]{.underline}×[\u3000 \u3000]{.underline}．\n\n5．（2分）在算式7□3÷76中，□填[\u3000 \u3000]{.underline}时，商一定是一位数；□填[\u3000 \u3000]{.underline}时，商一定是两位数．（每□填一个即可）．\n\n6．（1分）小红13分走路936米，速度是[\u3000 \u3000]{.underline}．\n\n7．（1分）小明坐在第8组的第5排，位置表示为（8，5）；笑笑在第3组的第6排，表示为[\u3000 \u3000]{.underline}．\n\n8．（2分）晓东的身份证号码是610104200807237527，他是[\u3000 \u3000]{.underline}年[\u3000 \u3000]{.underline}月[\u3000 \u3000]{.underline}日出生的．\n\n9．（3分）如果用正负数记录收支情况，爸爸今天的领到3600元记为[\u3000 \u3000]{.underline}元，充

In [20]:
result = sent_gpt_request(per_process(prompt))
result = result[result.find("["): result.rfind("]")+1].replace("\t","")

正在请求...


In [21]:
import json
import ast

print(prompt.replace("\n\n", "\n"))
ast.literal_eval(result)


**人教新版六年级下数学综合检测题（含解析）**
学校:\_\_\_\_\_\_\_\_\_\_\_姓名：\_\_\_\_\_\_\_\_\_\_\_班级：\_\_\_\_\_\_\_\_\_\_\_考号：\_\_\_\_\_\_\_\_\_\_\_
  ------ ---- ---- ---- ---- ---- ------
  题号   一   二   三   四   五   总分
  得分                            
  ------ ---- ---- ---- ---- ---- ------
**一．选择题（共25小题）**
1．如果把学校的位置记作0*m*，从学校出发向东走300*m*有一个书店，书店的位置记作+300*m*，从书店出发向西走600*m*有一个报亭，报亭的位置记作（ ）*m*．
A．+900 B．﹣300 C．﹣900 D．﹣600
2．在数轴上，0左边的数（ ）
A．是正数
B．是负数
C．既不是正数，也不是负数
3．下面的数与0最接近的一个数是（ ）
A．﹣5 B．﹣1 C．+5 D．+2
4．某市一月份的平均气温是﹣2℃，二月份的平均气温比一月份的升高了5℃，该市二月份的平均气温是（ ）
A．5℃ B．3℃ C．7℃ D．﹣5℃
5．下列算式中，结果与![](./notebook/image/media/image1.jpeg)不相等的是（ ）
A．0.2÷0.5 B．20÷500 C．4÷10 D．16÷40
6．已知"甲比乙多50%°则下面叙述中正确的是（ ）
A．甲与乙的比是2：3 B．乙比甲少50%
C．甲与乙的比是3：2 D．乙是甲的3倍
7．在![](./notebook/image/media/image2.jpeg)＝0.25＝（ ）÷12＝（ ）%中所填数据完全正确的是（ ）
A．5，3，25 B．5，25，3 C．25，3，5 D．3，5，25
8．在60后面添上百分号，这个数就（ ）
A．扩大100倍 B．缩小10倍
C．缩小为原来的1% D．大小不变
9．做一个圆柱形通风管要用多少铁皮是求（ ）
A．表面积 B．体积 C．底面积 D．侧面积
10．等底等高的长方体、正方体、圆柱的体积相比较．（ ）
A．长方体体积大 B．正方体体积大
C．圆柱体积大 D．一样大
11．一个圆

[{'answer': 'B',
  'option': {'A': '+900', 'B': '﹣300', 'C': '﹣900', 'D': '﹣600'},
  'question': '1．如果把学校的位置记作0m，从学校出发向东走300m有一个书店，书店的位置记作+300m，从书店出发向西走600m有一个报亭，报亭的位置记作（ ）m．',
  'explanation': ''},
 {'answer': 'B',
  'option': {'A': '是正数\n', 'B': '是负数\n', 'C': '既不是正数，也不是负数\n'},
  'question': '2．在数轴上，0左边的数（ ）',
  'explanation': ''},
 {'answer': 'B',
  'option': {'A': '﹣5', 'B': '﹣1', 'C': '+5', 'D': '+2'},
  'question': '3．下面的数与0最接近的一个数是（ ）',
  'explanation': ''},
 {'answer': 'B',
  'option': {'A': '5℃', 'B': '3℃', 'C': '7℃', 'D': '﹣5℃'},
  'question': '4．某市一月份的平均气温是﹣2℃，二月份的平均气温比一月份的升高了5℃，该市二月份的平均气温是（ ）',
  'explanation': ''},
 {'answer': 'B',
  'option': {'A': '0.2÷0.5', 'B': '20÷500', 'C': '4÷10', 'D': '16÷40'},
  'question': '5．下列算式中，结果与![](./notebook/image/media/image1.jpeg)不相等的是（ ）',
  'explanation': ''},
 {'answer': 'C',
  'option': {'A': '甲与乙的比是2：3',
   'B': '乙比甲少50%\n',
   'C': '甲与乙的比是3：2',
   'D': '乙是甲的3倍'},
  'question': '6．已知"甲比乙多50%°则下面叙述中正确的是（ ）',
  'explanation': ''},
 {'

# 跑一些数据

In [ ]:
import json
from pathlib import Path

save_file_name = "topics.jsonl"

completed_file = set()

if Path(save_file_name).exists():
    with open(save_file_name, "r") as f:
        for line in f:
            completed_topic = json.loads(line)
            completed_file.add(completed_topic["file_path"])


for row in examination_paper_list:
    batchs = generate_batch(row["text"])
    if len(batchs) != 1:
        continue
    
    if row["file_path"] in completed_file:
        continue
    
    result = sent_gpt_request(batchs[0])
    result = result[result.find("["): result.rfind("]")+1].replace("\t","")
    topics = ast.literal_eval(result)
    
    with open(save_file_name, "a") as f:
        for topic in topics:
            topic.update({"file_path": str(row["file_path"])})
            f.write(json.dumps(topic) + "\n")
            
        completed_file.add(row["file_path"])
    

正在请求...
正在请求...
请求失败重新请求中... Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Fri, 21 Jul 2023 15:09:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7ea453b5dcb80461-HKG', 'alt-svc': 'h3=":443"; ma=86400'}


In [36]:
"""'\n**2019-2020学年陕西省西安市陕师大附小四年级（上）期末数学试卷**\n\n**一、填空题（共22分）**\n\n1．（2分）2017年国庆黄金周，西安共接待游客11650300人次，横线上的数读作[\u3000 \u3000]{.underline}；旅游收入5795000000元人民币，四舍五入到亿位大约是[\u3000 \u3000]{.underline}亿元．\n\n2．（2分）黑板的上下两条边互相[\u3000 \u3000]{.underline}；相邻的两条边互相[\u3000 \u3000]{.underline}．\n\n3．（1分）钟面上[\u3000 \u3000]{.underline}时整的时候，时针和分针成平角．\n\n4．（4分）（*a*+*b*）+*c*＝[\u3000 \u3000]{.underline}+（[\u3000 \u3000]{.underline}+[\u3000 \u3000]{.underline}）；（*a*+*b*）×*c*＝[\u3000 \u3000]{.underline}×[\u3000 \u3000]{.underline}+[\u3000 \u3000]{.underline}×[\u3000 \u3000]{.underline}．\n\n5．（2分）在算式7□3÷76中，□填[\u3000 \u3000]{.underline}时，商一定是一位数；□填[\u3000 \u3000]{.underline}时，商一定是两位数．（每□填一个即可）．\n\n6．（1分）小红13分走路936米，速度是[\u3000 \u3000]{.underline}．\n\n7．（1分）小明坐在第8组的第5排，位置表示为（8，5）；笑笑在第3组的第6排，表示为[\u3000 \u3000]{.underline}．\n\n8．（2分）晓东的身份证号码是610104200807237527，他是[\u3000 \u3000]{.underline}年[\u3000 \u3000]{.underline}月[\u3000 \u3000]{.underline}日出生的．\n\n9．（3分）如果用正负数记录收支情况，爸爸今天的领到3600元记为[\u3000 \u3000]{.underline}元，充话费用去100元，记为[\u3000 \u3000]{.underline}，读作[\u3000 \u3000]{.underline}．\n\n10．（4分）在下面的空格中填入"＜""＝""＞"\n\n> 637058[\u3000 \u3000]{.underline}4328500\n>\n> 160000000[\u3000 \u3000]{.underline}16亿\n>\n> 180÷12[\u3000 \u3000]{.underline}180÷15\n>\n> 150×2[\u3000 \u3000]{.underline}15×20\n\n**二、选择题（10分，每题2分）**\n\n11．（2分）一个八位数，它的最高位是（\u3000\u3000）\n\nA．百万位 B．千万位 C．亿位 D．十亿位\n\n12．（2分）六十三亿零二十万五千零一，写作（\u3000\u3000）\n\nA．630205001 B．630020500001\n\nC．6300205001 D．63020501\n\n13．（2分）下面的数，一个0也不读的是（\u3000\u3000）\n\nA．80080000 B．80008000 C．80000800 D．80000080\n\n14．（2分）与4200÷70的商相同的是（\u3000\u3000）\n\nA．42÷7 B．420÷7 C．4200÷700 D．42000÷70\n\n15．（2分）与63×99计算结果相等的是（\u3000\u3000）\n\nA．63×99+1 B．63×100﹣1 C．63×99+63 D．63×100﹣63\n\n**三、判断题（10分，每题2分）**\n\n16．（2分）四舍五入得到的近似数可能比这个数大，可能比这个数小．[\u3000 \u3000]{.underline}（判断对错）\n\n17．（2分）经过一点可以画一条直线，经过两点可以画两条直线．[\u3000 \u3000]{.underline}（判断对错）\n\n18．（2分）试商时，如果余数比除数大，应把商改大．[\u3000 \u3000]{.underline}．（判断对错）\n\n19．（2分）0是正数，不是负数．[\u3000 \u3000]{.underline}．（判断对错）\n\n20．（2分）零下9度比零上6度的温度高．[\u3000 \u3000]{.underline}（判断对错）\n\n**四、计算（25分）**\n\n21．（5分）直接写得数．\n\n  80÷20＝      25+40＝   670+140＝   4800÷600＝   3500÷70＝\n  ------------ --------- ----------- ------------ -----------\n  930﹣250＝   15×60＝   320÷40＝    297×4＝      810÷90＝\n\n22．（8分）列竖式计算\n\n> 396×27＝\n>\n> 208×34＝\n>\n> 368÷46＝\n>\n> 1008÷28＝\n\n23．（12分）简便运算\n\n> 357+136+243\n>\n> 58×36+64×58\n>\n> 8×72×125\n>\n> 420×（945÷15﹣28）\n\n**五、操作题（13分）**\n\n24．（6分）536980和541508"四舍五入"后都是54万．\n\n> （1）在数轴上标出两个数的位置．\n>\n> （2）这两个数，[\u3000 \u3000]{.underline}更接近54万．\n\n25．（4分）画出这条直线的一条平行线，一条垂线．\n\n26．（3分）以这条射线为角的一边，画出125度的角，标出度数．\n\n**六、解决问题（20分）**\n\n27．（5分）6支球队比赛，两两比一场，一共要比多少场？\n\n> （1）请画出示意图．\n>\n> （2）有顺序的数一数，列式计算看一看．\n\n28．（5分）一辆货车从甲地去乙地，去时每时行48千米，用了18小时到了乙地，回来每时行54千米，需要用时多久才能回来呢？\n\n29．（5分）一张桌子152元，一把椅子48元，学校要买75套桌椅，一共多少钱呢？\n\n30．（5分）创建美丽西安，离不开清洁工人的努力，一名清洁工阿姨一个月清扫了2511千克垃圾，按这样计算，一年可以清扫多少千克的垃圾呢？\n\n**2019-2020学年陕西省西安市陕师大附小四年级（上）期末数学试卷**\n\n**参考答案与试题解析**\n\n**一、填空题（共22分）**\n\n1．【分析】根据整数的读法，从高位到低位，一级一级地读，每一级末尾的0都不读出来，其余数位连续几个0都只读一个零，即可读出此数；改写成用"亿"作单位的数，就是把千万位数字9四舍五入进1，保留整亿．\n\n> 【解答】解：11650300读作：一千一百六十五万零三百；\n>\n> 5795000000≈58亿；\n>\n> 故答案为：一千一百六十五万零三百，58．\n>\n> 【点评】本题主要考查整数的读、写法、改写和求近似数，注意改写和求近似数时要带计数单位．\n\n2．【分析】因为黑板是一个长方形，所以根据长方形的特征：对边平行且相等，4个角都是直角，可知，长方形相邻的两条边互相垂直，相对的两边互相平行；据此解答．\n\n> 【解答】解：据分析可知：\n>\n> 黑板的上下两条边互相 平行；相邻的两条边互相 垂直．\n>\n> 故答案为：平行、垂直．\n>\n> 【点评】解答此题的主要依据是：长方形的邻边互相垂直，对边互相平行．\n\n3．【分析】钟面一周为360°，共分12大格，每格为360÷12＝30°，6时整，分针与时针相差6个整大格，所以钟面上时针与分针形成的夹角是：30°×6＝180°，由此根据平角的定义即可解答．\n\n> 【解答】解：30°×6＝180°\n>\n> 180°的角是平角\n>\n> 6时整，钟面上的分针和时针所夹的角是平角；\n>\n> 故答案为：6．\n>\n> 【点评】抓住钟面上每个大格所对的夹角的度数是30度，找出时针与分针的夹角是几个格，即可计算解答．\n\n4．【分析】（1）根据加法结合律填空；\n\n> （2）根据乘法分配律填空．\n>\n> 【解答】解：（*a*+*b*）+*c*＝*a*+（ *b*+*c*）；\n>\n> （*a*+*b*）×*c*＝*a*×*c*+*b*×*c*．\n>\n> 故答案为：*a*，*b*，*c*；*a*，*c*，*b*，*c*．\n>\n> 【点评】加法结合律：先把前两个数相加，或先把后两个数相加，和不变．字母表示：*a*+*b*+*c*＝*a*+（*b*+*c*）\n>\n> 乘法分配律：两个数的和与一个数相乘，可以把它们与这个数分别相乘再相加，字母表示：*a*×（*b*+*c*）＝*ab*+*ac*．\n\n5．【分析】7□3÷76，被除数的前两位数7□小于除数76，所得的商是一位数；被除数的前两位数7□大于或等于除数76，所得的商是两位数；然后再进一步解答．\n\n> 【解答】解：7□3÷76中，要使商是一位数，7□＜76，□里面可以填0、1、2、3、4、5；\n>\n> 要使商是两位数，7□≥76，□里面可以填6、7、8、9；\n>\n> 故答案为：0，6（答案不唯一）．\n>\n> 【点评】三位数除以两位数，被除数的前两位数小于除数，所得的商是一位数；被除数的前两位数大于或等于除数，所得的商是两位数．\n\n6．【分析】根据路程÷时间＝速度，用小红13分钟走的路程除以用的时间，求出速度是多少，据此计算即可解答．\n\n> 【解答】解：936÷13＝72（米/分）\n>\n> 答：她的速度是72米/分．\n>\n> 故答案为：72米/分．\n>\n> 【点评】此题主要考查了行程问题中速度、时间和路程的关系：速度×时间＝路程，路程÷时间＝速度，路程÷速度＝时间，要熟练掌握．\n\n7．【分析】由"小明坐在第8组的第5排，位置表示为（8，5）"可知数对中第一个数字表示组，第二个数字表示排，据此即可用数对表示出笑笑坐的位置．\n\n> 【解答】解：小明坐在第8组的第5排，位置表示为（8，5）；笑笑在第3组的第6排，表示为（3，6）．\n>\n> 故答案为：（3，6）．\n>\n> 【点评】解答此题的关键是根据已知条件确定数对中每个数字所表示的意义．\n\n8．【分析】身份证的第7～14位表示出生日期，其中第7～10位是出生的年份，11、12位是出生的月份，第13、14位是出生日；据此解答．\n\n> 【解答】解：610104200807237527这个身份证号码的7～14位是20080723，所以晓东是 2008年 7月 23日出生的．\n>\n> 故答案为：2008，7，23．\n>\n> 【点评】本题是考查身份证的数字编码问题，身份证上：\n>\n> 1、前六位是地区代码；\n>\n> 2、7～14位是出生日期；\n>\n> 3、15～17位是顺序码，其中第17位奇数分给男性，偶数分给女性；\n>\n> 4、第18位是校验码．\n\n9．【分析】此题主要用正负数来表示具有意义相反的两种量：领取钱数记为正，则花费钱数就记为正，由此直接得出结论即可；根据负数的读法，读出即可．\n\n> 【解答】解：如果用正负数记录收支情况，爸爸今天的领到3600元记为+3600元，充话费用去100元，记为﹣100元，读作 负一百元．\n>\n> 故答案为：+3600，﹣100元，负一百元．\n>\n> 【点评】此题主要考查正负数的意义，正数与负数表示意义相反的两种量，看清规定哪一个为正，则和它意义相反的就为负．\n\n10．【分析】（1）（2）按照整数比较大小的方法判断即可；\n\n> （3）被除数不变，除数越大，则商越小，据此判断即可；\n>\n> （4）根据积不变的规律判断即可．\n>\n> """.split("参考答案与试题解析")[1]

'**\n\n**一、填空题（共22分）**\n\n1．【分析】根据整数的读法，从高位到低位，一级一级地读，每一级末尾的0都不读出来，其余数位连续几个0都只读一个零，即可读出此数；改写成用"亿"作单位的数，就是把千万位数字9四舍五入进1，保留整亿．\n\n> 【解答】解：11650300读作：一千一百六十五万零三百；\n>\n> 5795000000≈58亿；\n>\n> 故答案为：一千一百六十五万零三百，58．\n>\n> 【点评】本题主要考查整数的读、写法、改写和求近似数，注意改写和求近似数时要带计数单位．\n\n2．【分析】因为黑板是一个长方形，所以根据长方形的特征：对边平行且相等，4个角都是直角，可知，长方形相邻的两条边互相垂直，相对的两边互相平行；据此解答．\n\n> 【解答】解：据分析可知：\n>\n> 黑板的上下两条边互相 平行；相邻的两条边互相 垂直．\n>\n> 故答案为：平行、垂直．\n>\n> 【点评】解答此题的主要依据是：长方形的邻边互相垂直，对边互相平行．\n\n3．【分析】钟面一周为360°，共分12大格，每格为360÷12＝30°，6时整，分针与时针相差6个整大格，所以钟面上时针与分针形成的夹角是：30°×6＝180°，由此根据平角的定义即可解答．\n\n> 【解答】解：30°×6＝180°\n>\n> 180°的角是平角\n>\n> 6时整，钟面上的分针和时针所夹的角是平角；\n>\n> 故答案为：6．\n>\n> 【点评】抓住钟面上每个大格所对的夹角的度数是30度，找出时针与分针的夹角是几个格，即可计算解答．\n\n4．【分析】（1）根据加法结合律填空；\n\n> （2）根据乘法分配律填空．\n>\n> 【解答】解：（*a*+*b*）+*c*＝*a*+（ *b*+*c*）；\n>\n> （*a*+*b*）×*c*＝*a*×*c*+*b*×*c*．\n>\n> 故答案为：*a*，*b*，*c*；*a*，*c*，*b*，*c*．\n>\n> 【点评】加法结合律：先把前两个数相加，或先把后两个数相加，和不变．字母表示：*a*+*b*+*c*＝*a*+（*b*+*c*）\n>\n> 乘法分配律：两个数的和与一个数相乘，可以把它们与这个数分别相乘再相加，字母表示：*a*×（*b*+*c*）＝*ab*+*ac*．\n\n5．【分析】7□3÷76